## [CPE312] Group Assignment
## การสกัดและวิเคราะห์ข้อมูลวิดีโอจาก "YouTube Channels"

บริษัทสตาร์ทอัพด้านสื่อดิจิทัลต้องการทำความเข้าใจ **แนวโน้มความนิยมของคอนเทนต์บน YouTube** เพื่อใช้วางกลยุทธ์การสร้างวิดีโอใหม่ให้ตรงกับความสนใจของผู้ชมมากที่สุด โดยเน้นการวิเคราะห์ **จำนวนการเข้าชม (views), ความยาววิดีโอ (duration), วันที่เผยแพร่ (published date)** และ **ประเภทของคอนเทนต์**

<p>วัตถุประสงค์:</p>

1. ให้นิสิตจับกลุ่ม กลุ่มละ 3–4 คน  
2. เพื่อสกัดข้อมูลจาก YouTube Channel ที่กำหนด (เช่น Title, Views, Published Date, URL)  
3. เพื่อวิเคราะห์ว่า **ประเภทของวิดีโอใดได้รับความนิยมมากที่สุด** (วัดจากจำนวนการเข้าชม)  
4. เพื่อดูแนวโน้มการอัปโหลด (เช่น รายเดือน/รายสัปดาห์) และความสัมพันธ์กับจำนวนการเข้าชม  
5. เพื่อสรุปเป็น **ข้อเสนอเชิงกลยุทธ์** ว่า Content แบบใดควรผลิตมากขึ้นเพื่อเพิ่มโอกาสเข้าถึงผู้ชม  

<p>ผลลัพธ์ที่คาดหวัง:</p>

- Dataset ที่สกัดมาในรูปแบบ CSV/Excel พร้อม Clean ข้อมูลเบื้องต้น  
- Dashboard/Visualization ที่แสดงแนวโน้ม เช่น กราฟแท่ง, กราฟเส้น, พายชาร์ต  
- presentation สรุปขั้นตอนการสกัดข้อมูล, ผลการวิเคราะห์, และข้อเสนอเชิงกลยุทธ์  

<p>คำแนะนำ:</p>

- นิสิตสามารถใช้ Selenium/ YouTube API (ถ้ามี key) / Google chorme Extension ในการดึงข้อมูล  
- เน้นการ **อธิบาย insight** มากกว่าโค้ดที่ซับซ้อน  
- ให้แบ่งบทบาทในกลุ่ม (เช่น ผู้สกัดข้อมูล, ผู้วิเคราะห์, ผู้ทำ visualization, ผู้เขียนรายงาน) เพื่อฝึก teamwork จริง  


### Practice: Youtube Data Scarping using Selenium


### สิ่งที่ได้ muhaha
- ไฟล์ CSV: `youtube_videos.csv` มีคอลัมน์ `Title, Views, Duration, Published, URL, Type, Views_Count, Duration_Seconds`
- โค้ดตัวอย่างสำหรับ **ทำความสะอาดข้อมูลเบื้องต้น** และ **แปลงตัวเลขวิว/เวลา**

### ข้อควรรู้
- โค้ดนี้ใช้ **Selenium 4** ซึ่งมี Selenium Manager จัดการ ChromeDriver ให้โดยอัตโนมัติ (ควรมี Google Chrome ในเครื่อง)
- เนื้อหาบางส่วนของ YouTube เปลี่ยนแปลงได้ (โดยเฉพาะ Shorts) ระยะยาวควรพิจารณา YouTube Data API


## ขั้นที่ 1 — ติดตั้งไลบรารี 
รันในเทอร์มินัลของ VS Code 

```powershell
pip install selenium beautifulsoup4 lxml
```



In [ ]:
# pip install selenium beautifulsoup4 lxml

## ขั้นที่ 2 — ตั้งค่าช่อง YouTube ที่ต้องการดึงข้อมูล
แก้ตัวแปร `CHANNEL_URLS` ด้านล่างเป็นช่องที่ต้องการ (หน้า **/videos**) ได้ตามต้องการ

In [103]:
# ตั้งค่าช่อง YouTube (แก้ตามต้องการ)
CHANNEL_URLS = [
    "https://www.youtube.com/@lorndiary/videos",
]

# พารามิเตอร์การสกรอลล์หน้าเพื่อโหลดวิดีโอให้มากขึ้น
MAX_SCROLLS = 1500     # เพิ่มได้ถ้าอยากดึงมากขึ้น
SCROLL_PAUSE_SEC = 1.6
WAIT_TIMEOUT = 45

# ไฟล์ผลลัพธ์
CSV_PATH = "Scarping_หลอนไดอารี่.csv"


## ขั้นที่ 3 — ฟังก์ชันช่วยแปลงค่า (Views → ตัวเลข, Duration → วินาที)
รองรับหน่วยภาษาอังกฤษ (K, M, B) และคำไทย (พัน, หมื่น, แสน, ล้าน) แบบประมาณการ

In [104]:
import re
import math
from typing import Optional

_views_re = re.compile(r"([\d\.,]+)\s*([KkMmBbล้านพันหมื่นแสน]?)")

def parse_views_to_number(s: str) -> Optional[int]:
    if not s:
        return None
    s = s.replace(",", "").strip()

    m = _views_re.search(s)
    if not m:
        return None

    num_str, suffix = m.group(1), m.group(2)
    try:
        val = float(num_str)
    except ValueError:
        return None

    # อังกฤษ
    if suffix.lower() == "k":
        val *= 1_000
    elif suffix.lower() == "m":
        val *= 1_000_000
    elif suffix.lower() == "b":
        val *= 1_000_000_000

    # ไทย
    if "พัน" in suffix:
        val *= 1_000
    elif "หมื่น" in suffix:
        val *= 10_000
    elif "แสน" in suffix:
        val *= 100_000
    elif "ล้าน" in suffix:
        val *= 1_000_000

    return int(val) if not math.isnan(val) else None

def duration_to_seconds(text: str) -> Optional[int]:
    """ แปลงเวลาแบบ H:MM:SS หรือ M:SS → วินาที """
    if not text:
        return None
    parts = text.strip().split(":")
    try:
        parts = list(map(int, parts))
    except ValueError:
        return None
    if len(parts) == 3:
        h, m, s = parts
        return h*3600 + m*60 + s
    if len(parts) == 2: 
        m, s = parts
        return m*60 + s
    if len(parts) == 1:
        return parts[0]
    return None


## ขั้นที่ 4 — ดึงข้อมูลด้วย Selenium
- เลือกทีละ "การ์ดวิดีโอ" เพื่อให้ Title/Views/Duration/Published/URL **ไม่หลุดกัน**
- พยายามอ่าน Duration หลายรูปแบบ (Shorts อาจไม่มี) และจัดประเภท `Type`

In [105]:
import re
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

opts = Options()
opts.add_argument("--lang=th-TH")
# opts.add_argument("--headless=new")  # ถ้าต้องการรันแบบไม่เปิดหน้าต่าง # --lang=th-TH
driver = webdriver.Chrome(options=opts)
rows = []

#------------------Views → ตัวเลข-------------------------
_views_re = re.compile(r'([0-9\.]+)\s*(k|m|b|พัน|หมื่น|แสน|ล้าน)?')
def parse_views_to_number(s: str) -> Optional[int]:
    if not s:
        return None
    s = s.replace(",", "").strip()

    m = _views_re.search(s)
    if not m:
        return None

    num_str, suffix = m.group(1), m.group(2)
    try:
        val = float(num_str)
    except ValueError:
        return None
    
    if suffix:
        suffix = suffix.strip().lower()
    # อังกฤษ
    if suffix == "k":
        val *= 1_000
    elif suffix == "m":
        val *= 1_000_000
    elif suffix == "b":
        val *= 1_000_000_000
    # ไทย
    elif suffix in ["พัน"]:
        val *= 1_000
    elif suffix in ["หมื่น"]:
        val *= 10_000
    elif suffix in ["แสน"]:
        val *= 100_000
    elif suffix in ["ล้าน"]:
        val *= 1_000_000

    return int(val) if not math.isnan(val) else None
#------------------Duration → วินาที-------------------------
def duration_to_seconds(text: str) -> Optional[int]:
    """ แปลงเวลาแบบ H:MM:SS หรือ M:SS → วินาที """
    if not text:
        return None
    parts = text.strip().split(":")
    try:
        parts = list(map(int, parts))
    except ValueError:
        return None
    if len(parts) == 3:
        h, m, s = parts
        return h*3600 + m*60 + s
    if len(parts) == 2: 
        m, s = parts
        return m*60 + s
    if len(parts) == 1:
        return parts[0]
    return None

#------------------ดึงข้อมูลด้วย Selenium------------------------

try:
    for url in CHANNEL_URLS:
        driver.get(url)

        WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ytd-rich-item-renderer"))
        )

        # เลื่อนหน้าจอเพื่อโหลดวิดีโอเพิ่ม
        last_count = 0
        for _ in range(MAX_SCROLLS):
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(SCROLL_PAUSE_SEC)
            cards_now = driver.find_elements(By.CSS_SELECTOR, "ytd-rich-item-renderer")
            if len(cards_now) <= last_count:
                break
            last_count = len(cards_now)

        cards = driver.find_elements(By.CSS_SELECTOR, "ytd-rich-item-renderer")
        
        for c in cards:
            title = views_raw = published = url_ = ""

            # Title + URL
            try:
                a = c.find_element(By.CSS_SELECTOR, "a#video-title-link, a#video-title")
                title = (a.get_attribute("title") or a.text).strip()
                url_ = a.get_attribute("href") or ""
            except Exception:
                pass

            # Views + Published
            try:
                metas = c.find_elements(By.CSS_SELECTOR, "span.inline-metadata-item")
                if len(metas) >= 1:
                    views_raw = metas[0].text.strip()
                if len(metas) >= 2:
                    published = metas[1].text.strip()
            except Exception:
                pass
            # Duration
            duration_text = ""
            duration_sec = None
            try:
                duration_elem = c.find_element(
                    By.XPATH, './/ytd-thumbnail-overlay-time-status-renderer//span'
                )
                duration_text = duration_elem.get_attribute("innerText").strip()
                duration_sec = duration_to_seconds(duration_text)
            except Exception:
                duration_text = ""
                duration_sec = None
            # แปลงค่าตัวเลข
            views_num = parse_views_to_number(views_raw)
            if title:
                rows.append({
                    "Title": title,
                    "Views": views_raw,
                    "Views_Count": views_num if views_num is not None else "",
                    "Published": published,
                    "URL": url_,
                    "Duration_Text": duration_text,
                    "Duration_Seconds": duration_sec if duration_sec is not None else ""
                })
finally:
    driver.quit()

print(f"ดึงมาได้ {len(rows)} แถว")



ดึงมาได้ 564 แถว


## ขั้นที่ 5 — บันทึกเป็น CSV (UTF-8 with BOM เพื่อให้ Excel เปิดภาษาไทยถูก)


In [106]:
fieldnames = [
    "Title", "Views", "Views_Count", 
    "Published", "URL","Duration_Text","Duration_Seconds"
]
with open(CSV_PATH, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(rows)

print(f"บันทึกไฟล์: {CSV_PATH}")


บันทึกไฟล์: Scarping_หลอนไดอารี่.csv


## ขั้นที่ 6 — (ตัวอย่าง) ดูพรีวิวข้อมูลด้วย pandas
_ส่วนนี้ไม่จำเป็น แต่ช่วยเช็กเร็ว ๆ ว่าข้อมูลโอเค_

In [107]:
import pandas as pd

# โหลด CSV
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# เรียงลำดับตามยอดวิว (Views_Count) จากมากไปน้อย
if "Views_Count" in df.columns:
    df_sorted = df.sort_values(by="Views_Count", ascending=False)
    print(df_sorted.head(15))
else:
    print("ไม่พบคอลัมน์ Views_Count ในไฟล์ CSV")


                                                 Title                Views  \
515                   กระสือยายสาย | หลอนไดอารี่ EP.45  การดู 35 ล้าน ครั้ง   
525                 ตำนานหงส์ยักษ์ | ตำนานไดอารี่ EP.2  การดู 21 ล้าน ครั้ง   
529                  ผีกระหังตาก่ำ | หลอนไดอารี่ EP.34  การดู 17 ล้าน ครั้ง   
509                   ตายแล้วไปไหน | หลอนไดอารี่ EP.48  การดู 14 ล้าน ครั้ง   
476                   รถไฟขบวนหลอน | หลอนไดอารี่ EP.70  การดู 13 ล้าน ครั้ง   
382  ศึกคุณไสย สงครามจอมขมังเวทย์ | หลอนไดอารี่ EP.123  การดู 13 ล้าน ครั้ง   
420                    ผีกินลูกคน | หลอนไดอารี่ EP.105  การดู 13 ล้าน ครั้ง   
519           พระอุ้มหมา ชีอุ้มแมว | หลอนไดอารี่ EP.42  การดู 13 ล้าน ครั้ง   
422            ศาลาพักข้างสุสาน 1 | หลอนไดอารี่ EP.104  การดู 13 ล้าน ครั้ง   
534              ผีม้าบ้อง อาละวาด | หลอนไดอารี่ EP.29  การดู 12 ล้าน ครั้ง   
501                     ผีสาวโกสุม | หลอนไดอารี่ EP.51  การดู 12 ล้าน ครั้ง   
507             ปอบตาหอม คืนอยู่ไฟ | หลอนไดอารี่ EP.